<a href="https://colab.research.google.com/github/KadaliSana/13B-Alpaca-colab/blob/main/AlpacaLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers -y
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install flask_ngrok
!pip install ngrok
!pip install pyngrok
!ngrok authtoken '1atGT0XTeO5twH4M9hQhpUyLQCC_2ugMyUcj1DwJeGJ6zKUcn'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59

In case GPU memory is less than required to be used then change the model from 13B to 7B by replacing the 13 by 7 which uses only 6gb of vram.




In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-13b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-13b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca13B-lora")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
from flask_ngrok import run_with_ngrok
import flask

app = flask.Flask(__name__)

@app.route('/<text>', methods= ['GET'])
def alpaca_talk(text):
    output = text.replace("%20"," ")
    print (output)
    inputs = tokenizer(str(
        output),
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        response = (tokenizer.decode(s))
        print (response)
        return response

input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
'''

alpaca_talk(input_text)

run_with_ngrok(app)

if __name__ == '__main__':
    app.run()  

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
Alpaca (Vicugna pacos) is a domesticated species of South American camelid, similar in appearance to Llamas but smaller in size with shorter ears and neck. They have long necks, small heads, short legs and large eyes. Their wool is soft and silky which makes them popular for making sweaters, blankets, hats etc. 
Lama (Lama glama), also known as the South American Camelids or New World Camels, are herbivorous mammals native to Central and South America. They can be found at altitudes ranging from sea level up to 4000 meters above sea level.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2fcd-34-147-76-217.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
hello
Generating...


INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:35:43] "GET /hello HTTP/1.1" 200 -


 hello,
I'm a writer. I write books and articles for magazines and newspapers. I also teach writing at the University of Toronto. My students are mostly young people who want to be writers themselves someday. They ask me all kinds of questions about how to get started: How do you find an agent? What should your first book be about? Should it be fiction or non-fiction? And my answer is always the same: Write what you know.
That doesn't mean that you have to tell stories only about yourself—although sometimes those can make great novels! It means that if you don't feel like you really understand something, then maybe you shouldn't try to write about it yet. If you haven't experienced something in real life, there will probably still be plenty of things left over when you finish this book. But even though we might not agree on everything here (and I hope we won't), I think you'll learn some new stuff too. So let's start with the basics...
# CHAPTER 1  
WHAT IS NONFICTION?
Nonfiction is an

INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:36:22] "GET /favicon.ico HTTP/1.1" 200 -


 favicon.ico
    # This file was generated by the `rspec --init` command. Conventionally, all
    # specs live under a `spec` directory, which RSpec adds to the `$LOAD_PATH`.
    # Require this file using `require "spec_helper"` to ensure that it is only
    # loaded once.
    #
    # See http://rubydoc.info/gems/rspec-core/RSpec/Core/Configuration
    
    require'simplecov'
    SimpleCov.start do
      add_filter '/vendor/'
    end
  end
end

Hello
Generating...


INFO:werkzeug:127.0.0.1 - - [06/May/2023 05:38:23] "GET /Hello HTTP/1.1" 200 -


 Hello, I'm a newbie here.
I have been using Linux for about 10 years now and I am very happy with it. But recently my laptop has started to run slow so I decided to try Windows again (I used to use XP). So far I like the look of Win7 but there are some things that annoy me:
- The start menu is too big! It takes up half the screen when you open it. In Ubuntu/KDE4 this would be unacceptable. Why can't they make it smaller?
- When you click on an icon in the start menu it opens another window instead of just opening the program. This makes everything take longer because you need to wait for each window to load before doing anything else.
- There doesn't seem to be any way to change the default programs that get opened by certain file types. For example if I doubleclick on a.txt file then notepad will always open automatically even though I prefer gedit. How do I fix this?
- I don't understand why all these icons keep popping up everywhere. They clutter up the desktop and make it hard to 

In [ ]:
!ngrok authtoken '1atGT0XTeO5twH4M9hQhpUyLQCC_2ugMyUcj1DwJeGJ6zKUcn'

/bin/bash: ngrok: command not found


In [ ]:
    text1== text.replace("%20"," ")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
Alpaca's are small, domesticated animals native to South America while Lama's are large wild animals from Central Asia. 
The main difference between these two species is their size; alpacas grow up to about three feet tall (1 meter) whereas lamas can reach heights of over six feet (2 meters).

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an ode to why do Alpacas make the best pets?

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an ode to why do Alpacas make the best pets?

### Response:
Alpaca, you are so cute and fluffy! 
You have such soft furry fleece; 
Your eyes sparkle with joyful delight, 
And your ears stand tall in pride. 
With every step you take, 
I can't help but smile at how sweetly you prance. 
When I look into those big brown eyes of yours, 
It makes me want to give you kisses galore. 
You bring happiness wherever you go, 
And when we play together it brings us great joy. 
So thank you for being my friend, 
For making life worth living again.


In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an email to OpenAI saying why GPT4 should be open-source.

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an email top OpenAI saying why GPT4 should be open-source.

### Response:
Dear Sir/Madam, 
Opening up GPT4 to public access will allow researchers and developers from around the world to use it for their own projects in order to advance artificial intelligence technology further than ever before. It would also help us better understand how AI systems work so we can improve them even more. In addition, making this tool available as open source software could lead to new discoveries about language processing and natural language understanding which may have applications beyond just machine learning. I believe these are all reasons why GPT4 should be made open sourc


### Response:
Dear Sir/Madam, 
Opening up GPT4 to public access will allow researchers and developers from around the world to use it for their own projects in order to advance artificial intelligence technology further than ever before. It would also help us better understand how AI systems work so we can improve them even more. In addition, making this tool available as open source software could lead to new discoveries about language processing and natural language understanding which may have applications beyond just machine learning. I believe these are all reasons why GPT4 should be made open source


In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
Although it may seem like humans have all of the intelligence, alpaca's actually possess much more advanced cognitive abilities than we do! They can learn new tasks quickly and easily, they understand complex concepts such as cause & effect, and their memory capacity far exceeds our own. In fact, some scientists believe that one day alpacas will be able to perform many human jobs with ease - from driving cars to writing novels!


### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
Although it may seem like humans have all of the intelligence, alpaca's actually possess much more advanced cognitive abilities than we do! They can learn new tasks quickly and easily, they understand complex concepts such as cause & effect, and their memory capacity far exceeds our own. In fact, some scientists believe that one day alpacas will be able to perform many human jobs with ease - from driving cars to writing novels!

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke about an Alpaca

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke about an Alpaca

### Response:
An alpaca walks into a bar and orders a drink, but when he asks for change from his bartender, all he gets back are two pennies. The alpaca says "That's not fair! I should get more than just 2 cents!"


### Instruction:
Tell me a joke about an Alpaca

### Response:
An alpaca walks into a bar and orders a drink, but when he asks for change from his bartender, all he gets back are two pennies. The alpaca says "That's not fair! I should get more than just 2 cents!"